##### Tentativa 4

### Imports

In [1]:
# Basic Imports
import tweepy
import pandas as pd
import geopandas as gpd
import numpy as np 
import time
import os
import re

# Plotting and Visualization
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
%matplotlib inline

# TextBlob Imports
from textblob import TextBlob
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

# NLTK Imports
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

#MAP
from geopy.geocoders import Nominatim
import folium
from folium import plugins
from geopy.geocoders import Nominatim

### Chaves de autenticação

In [2]:
#Keys for autentication
consumer_key='TZ4vEqyiRbYVtm4vx71lI8eAP'
consumer_secret='mtD7gxLQCj96YYBBUbGxYdYogbbMLwQG1mSGQkHpeJZEKfkLu2'

access_token='859751370798379009-h4k3AsrvfxBSLBX0kowYjz42zPCJw9t'
access_token_secret='4jsA9noFsVhBnleJqYnd3DyILKpzHh8Xt9iex4ZFE4bMQ'

### Autenticação Twitter

In [3]:
#Autentication Methods
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

### Procurando Tweets

In [4]:

#Searchin Twitter Timelines
tweets = []
info = []

for tweet in tweepy.Cursor(api.search,
                           q="corona OR covid OR coronavírus OR covid19 OR covid-19 OR sars",
                           tweet_mode='extended',
                           rpp=100,
                           result_type="popular",
                           include_entities=True,
                           lang="pt").items(20000000):
    if 'retweeted_status' in dir(tweet):
        aux=tweet.retweeted_status.full_text
    else:
        aux=tweet.full_text
        
    newtweet = aux.replace("\n", " ")
   
    tweets.append(newtweet)
    info.append(tweet)
    
    file = open("covid.txt", "a", -1, "utf-8")
    file.write(newtweet+'\n')
    file.close()
    
    time.sleep(0.5)

### Criando o dataframe

In [5]:
#Construction of the dataframe
tweets_df = pd.DataFrame(tweets, columns=['Tweets']) 

tweets_df['len']  = np.array([len(tweet) for tweet in tweets])
tweets_df['ID']   = np.array([tweet.id for tweet in info])
tweets_df['Date'] = np.array([tweet.created_at for tweet in info])
tweets_df['Source'] = np.array([tweet.source for tweet in info])
tweets_df['Likes']  = np.array([tweet.favorite_count for tweet in info])
tweets_df['RTs']    = np.array([tweet.retweet_count for tweet in info])
tweets_df['User Location']    = np.array([tweet.user.location for tweet in info])
tweets_df['Geo']    = np.array([tweet.geo for tweet in info])
tweets_df['Coordinates']    = np.array([tweet.coordinates for tweet in info])

tweets_df.to_csv("covid.csv")

In [6]:
tweets_df.head()

,Tweets,len,ID,Date,Source,Likes,RTs,User Location,Geo,Coordinates
0,Não faz sentido excluir identificação biométri...,190,1283389477130571776,2020-07-15 13:14:30,Twitter for iPhone,10739,3263,Goiás,None,None
1,"Até aqui, estudamos a imunidade de quem se cur...",261,1283480075975958532,2020-07-15 19:14:30,Twitter Web App,21116,2780,São Paulo,None,None
2,"Entendo a sua revolta, faz todo sentido. Só ca...",284,1283788559846121473,2020-07-16 15:40:18,Tweetbot for Mac,19471,1715,São Paulo,None,None
3,o coronavirus vendo as pessoa sai sem mascara ...,69,1283591600237617152,2020-07-16 02:37:40,Twitter for iPhone,6805,2016,,None,None
4,"""Pior presidente do mundo no combate ao corona...",238,1283560152071901185,2020-07-16 00:32:42,Twitter for Advertisers,17163,4444,,None,None


### Top Tweets

In [7]:
fav_max = np.max(tweets_df['Likes'])
rt_max  = np.max(tweets_df['RTs'])

fav = tweets_df[tweets_df.Likes == fav_max].index[0]
rt  = tweets_df[tweets_df.RTs == rt_max].index[0]

# Max FAVs:
print("O tweet com mais curtidas é: \n{}".format(tweets_df['Tweets'][fav]))
print("Número de curtidas: {}".format(fav_max))
print("\n")
# Max RTs:
print("O tweet com mais retweet é: \n{}".format(tweets_df['Tweets'][rt]))
print("Número de retweets: {}".format(rt_max))

O tweet com mais curtidas é: 
Já saiu um estudo da estrutura 3D de um anticorpo neutralizando a proteína chave do coronavírus. Ou seja, já sabemos qual o melhor alvo pra vacinas. Isso é a ciência avançando absurdamente rápido na construção de uma vacina que funcione :)  https://t.co/SYqtdahC9m
Número de curtidas: 43969


O tweet com mais retweet é: 
Já saiu um estudo da estrutura 3D de um anticorpo neutralizando a proteína chave do coronavírus. Ou seja, já sabemos qual o melhor alvo pra vacinas. Isso é a ciência avançando absurdamente rápido na construção de uma vacina que funcione :)  https://t.co/SYqtdahC9m
Número de retweets: 6260


### Modelo de Treinamento

In [8]:
#Sentiment Analysis in Português and English

base_path = 'ReLi-Lex/'
train = []
wordsPT = []
wordsPT_sentiments = []

files = [os.path.join(base_path, f) for f in os.listdir(base_path)]

for file in files:
    t = 1 if '_Positivos' in file else -1
    with open(file, 'r', errors='ignore') as content_file:
        content = content_file.read()
        all = re.findall('\[.*?\]',content)
        for w in all:
            wordsPT.append((w[1:-1]))
            wordsPT_sentiments.append(t)
            train.append((w[1:-1], t))

cl = NaiveBayesClassifier(train)

In [9]:
def clean_tweet(tweet):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

# Análise de sentimento do Twitter com NLTK

### Modelo de treinamento

In [10]:
vectorizer = CountVectorizer(analyzer="word")
freq_tweets = vectorizer.fit_transform(wordsPT)
modelo = MultinomialNB()
modelo.fit(freq_tweets,wordsPT_sentiments);

### Análise de sentimento

In [11]:
tweetsarray = []

for tw in tweets_df['Tweets']:
    text = clean_tweet(tw)
    tweetsarray.append(text)

In [12]:
predictionData = vectorizer.transform(tweets_df['Tweets'])
tweets_df['SA NLTK'] = modelo.predict(predictionData)

In [13]:
#Sentiment Analysis Results
pos_tweets = [ tweet for index, tweet in enumerate(tweets_df['Tweets']) if tweets_df['SA NLTK'][index] > 0]
neg_tweets = [ tweet for index, tweet in enumerate(tweets_df['Tweets']) if tweets_df['SA NLTK'][index] < 0]

pos_locations = [ location for index, location in enumerate(tweets_df['User Location']) if tweets_df['SA NLTK'][index] > 0]
neg_locations = [ location for index, location in enumerate(tweets_df['User Location']) if tweets_df['SA NLTK'][index] < 0]

### Mapa dos Tweets

In [14]:

geolocator = Nominatim(user_agent="TweeterSentiments")

latitude_pos = []
longitude_pos = []

for user_location in pos_locations:
    try:
        location = geolocator.geocode(user_location)
        latitude_pos.append(location.latitude)
        longitude_pos.append(location.longitude)
    except:
        continue
        
latitude_neg = []
longitude_neg = []

for user_location in neg_locations:
    try:
        location = geolocator.geocode(user_location)
        latitude_neg.append(location.latitude)
        longitude_neg.append(location.longitude)
    except:
        continue

In [15]:
coordenadas_pos = np.column_stack((latitude_pos, longitude_pos))
coordenadas_neg = np.column_stack((latitude_neg, longitude_neg))

mapa = folium.Map(location=[-15.788497,-47.879873],zoom_start=4.)

mapa.add_child(plugins.HeatMap(coordenadas_pos, gradient={0:"gray", 1:"blue"}))
mapa.add_child(plugins.HeatMap(coordenadas_neg, gradient={0:"gray", 1:"red"}))
mapa.save('covid.html')
mapa